## Target Hypothesis

1. Score both Main & Co as two seperate customers

bs = fit(bs)

ws = fit(ws)




2. Make a model of both with Co as a dummy


bs = xtrain (bs,ws)


2.1 

    ws input 0


    if 1 Applicant:

        bs = xtrain (bs,0)

    else:

        bs = xtrain (bs,ws)


2.2  --> could be adversed selection due to that having a co applicant might say something about the relasionship between the two

    ws - applications with co -applicant and for these take the mean 


    if 1 Applicant:

        bs = xtrain (bs,mean(ws))

    else:

        bs = xtrain (bs,ws)




3. Worst 

if main = main:

else:

    ws




4. Best --makes sence     1. Main -- Always main -->  2. CO --> best score 


if main = main:

else:

    bs



5. Test two seperate scorecards for main & co

5.1 

     main = main 

     Co --> Best 

5.2

     main = main 

     Co --> CO 




In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import os
import sqlalchemy as sa
import pyodbc
import warnings
import urllib
warnings.filterwarnings('ignore')



################### GET PAYMENT DATA ##########################

path = "../1. Data/Cleaned Transaction Data"
os.chdir(path)


In [2]:
df = pd.read_excel('PreeProcessed_Psd2.xlsx')


In [3]:
df

,ReceivedDate,ApplicationID,AccountNumber,SSN,DisbursedDate,Amount,IsMainApplicant,ApplicantNo,HasCoapp,Ever90,...,Unclassified_partOfSalary9Months,Unclassified_sum12Months,Unclassified_sum3Months,Unclassified_sum6Months,Unclassified_sum9Months,sum creditors,sum debtCollectors,count creditors,count debtCollectors,_merge
0,2022-03-08,8205045,0,7711025523,2022-03-10,250000,1,1,0,1,...,50227.0,224.1,248.5,247.0,258.4,630444,81995,288551,519248,0
1,2022-04-07,8228347,0,9804048719,2022-04-22,400000,1,1,1,1,...,763.7,1333.3,1553.4,2678645.0,241491.0,1655157,2249127,0,0,0
2,2022-04-08,8229075,0,6501225061,2022-04-12,240000,1,1,0,1,...,85589.0,322.8,415.5,320.5,331.8,882268,279047,442904,685525,0
3,2022-04-11,8230481,0,9606236371,2022-04-13,295000,1,1,0,1,...,268.0,476.0,439.5,883993.0,100348.0,452428,627919,0,0,0
4,2022-04-15,8233936,0,8602147608,2022-06-08,270000,1,1,0,1,...,379.3,1362741.0,251386.0,547363.0,971946.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3488,2023-04-28,8660675,0,8902289209,2023-05-31,271000,1,1,1,0,...,195.3,381102.0,84024.0,274075.0,312523.0,0,0,0,0,0
3489,2023-04-28,8660675,0,8903213695,2023-05-31,271000,0,2,1,0,...,121.2,348937.0,91087.0,215240.0,265065.0,0,0,0,0,0
3490,2023-04-28,8660600,0,8007075586,2023-05-08,350000,1,1,1,0,...,40754.0,165.3,135.9,130.5,145.1,509094,147353,245526,367844,0
3491,2023-04-28,8660600,0,8203014934,2023-05-08,350000,0,2,1,0,...,38839.0,96.0,76.7,72.3,92.7,487593,113811,198074,353883,0


In [4]:
# pd.set_option('display.max_columns', None)

# git add .

# git commit -m "Finalised API Loop"

#  git status

#  git push origin Andreas

In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

class DataPreprocessor:
    def __init__(self, df):
        self.df = df

    def upsample(self, target_column):
        """
        Up-sample the minority class of a DataFrame based on the target column.

        Parameters:
        - target_column: string, the name of the target column

        Returns:
        - DataFrame with balanced classes
        """
        df_minority = self.df[self.df[target_column] == self.df[target_column].value_counts().idxmin()]
        df_majority = self.df[self.df[target_column] != self.df[target_column].value_counts().idxmin()]

        df_minority_upsampled = df_minority.sample(n=len(df_majority), replace=True, random_state=42)
        df_upsampled = pd.concat([df_majority, df_minority_upsampled], axis=0)
        
        return df_upsampled

    def from_quartile_idx(self, quartile):
        """
        Get the index of the quartile in df when creating train/test splits.

        Parameters:
        - quartile: float, the quartile to split on

        Returns:
        - position: int, the index position in the DataFrame
        """
        df = self.df.sample(frac=1).reset_index(drop=True)
        counter = 0
        eighty = np.round(df.Ever90.value_counts()[1] * quartile)
        position = 0

        for idx, i in enumerate(df.Ever90):
            if i == 1:
                counter += 1 
            if counter == eighty:
                position = idx
                print(position)
                break 

        print('Position in df cut:', position / len(df))
        return position

    def get_split_data_with_upsample_and_scaling(self, quartile1):
        """
        Get upsampled data where the split is based on a specified quartile to harmonize distribution,
        and then scale the data.

        Parameters:
        - quartile1: float, the quartile to split on

        Returns:
        - Tuple containing (Xtrain, Ytrain), (Xtest, Ytest), and train_upsampled
        """
        print('Binary Split: ' + str(self.df.Ever90.value_counts()))

        position1 = self.from_quartile_idx(quartile1)
        train, test = self.df.iloc[:position1], self.df.iloc[position1:]

        # Upsample only the training data
        train_upsampled = self.upsample('Ever90')
        print(train_upsampled.Ever90.value_counts())

        # Create Xtrain, Ytrain, Xtest, Ytest
        Xtrain, Ytrain = train_upsampled.drop(columns='Ever90'), train_upsampled['Ever90']
        Xtest, Ytest = test.drop(columns='Ever90'), test['Ever90']

        # Scale the data
        scaler = StandardScaler()
        Xtrain_scaled = scaler.fit_transform(Xtrain)
        Xtest_scaled = scaler.transform(Xtest)

        return (Xtrain_scaled, Ytrain.values), (Xtest_scaled, Ytest.values), train_upsampled


In [6]:
df = df.drop(columns=['ReceivedDate','ApplicationID','SSN','DisbursedDate','Amount','IsMainApplicant','ApplicantNo','NR'])

forforsta = df.drop(columns=['HasCoapp'])




In [7]:

preprocessor = DataPreprocessor(forforsta)
(Xtrain, Ytrain), (Xtest, Ytest), train_upsampled,  = preprocessor.get_split_data_with_upsample_and_scaling(0.7)

Binary Split: Ever90
0    3170
1     323
Name: count, dtype: int64
2434
Position in df cut: 0.696822215860292
Ever90
0    3170
1    3170
Name: count, dtype: int64


In [8]:
Xtrain

array([[ 0.        , -0.66024429, -0.03834957, ..., -0.14059477,
        -0.16299462,  0.        ],
       [ 0.        , -0.06464579, -0.03834957, ..., -0.6117184 ,
        -0.65450127,  0.        ],
       [ 0.        , -0.3724951 , -0.03834957, ..., -0.6117184 ,
        -0.65450127,  0.        ],
       ...,
       [ 0.        ,  0.66530619, -0.03834957, ..., -0.6117184 ,
        -0.65450127,  0.        ],
       [ 0.        , -0.61634138, -0.03834957, ...,  0.48079613,
         0.36468438,  0.        ],
       [ 0.        , -0.58195958, -0.03834957, ..., -0.6117184 ,
        -0.65450127,  0.        ]])

In [18]:
import numpy as np
import optuna
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

def objective(trial, model_name, X_train, y_train):
    if model_name == 'svm':
        C = trial.suggest_loguniform('C', 1e-3, 1e2)
        kernel = trial.suggest_categorical('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
        degree = trial.suggest_int('degree', 2, 5) if kernel == 'poly' else 3
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])
        model = SVC(C=C, kernel=kernel, degree=degree, gamma=gamma)
        
    elif model_name == 'xgboost':
        n_estimators = trial.suggest_int('n_estimators', 50, 200)
        max_depth = trial.suggest_int('max_depth', 3, 15)
        learning_rate = trial.suggest_loguniform('learning_rate', 0.01, 0.3)
        model = XGBClassifier(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate)
        
    elif model_name == 'random_forest':
        n_estimators = trial.suggest_int('n_estimators', 50, 200)
        max_depth = trial.suggest_int('max_depth', 5, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 4)
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)
        
    elif model_name == 'logistic_regression':
        C = trial.suggest_loguniform('C', 1e-3, 1e2)
        penalty = trial.suggest_categorical('penalty', ['l1', 'l2', 'elasticnet', 'none'])
        solver = trial.suggest_categorical('solver', ['lbfgs', 'liblinear', 'saga', 'sag'])
        model = LogisticRegression(C=C, penalty=penalty, solver=solver, max_iter=10000)

    scores = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy')
    return scores.mean()

def optimize_model(model_name, X_train, y_train, n_trials=2):
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(trial, model_name, X_train, y_train), n_trials=n_trials)
    return study.best_params

def train_best_model(model_name, best_params, X_train, y_train):
    if model_name == 'svm':
        model = SVC(**best_params)
    elif model_name == 'xgboost':
        model = XGBClassifier(**best_params)
    elif model_name == 'random_forest':
        model = RandomForestClassifier(**best_params)
    elif model_name == 'logistic_regression':
        model = LogisticRegression(**best_params, max_iter=10000)
    model.fit(X_train, y_train)
    return model

# Example usage:
# Assuming train and test data are numpy arrays
# Replace these with your actual data
# X_train, X_test, y_train, y_test = np.array(...)


In [19]:

models = ['svm', 'xgboost', 'logistic_regression', 'random_forest']
best_params = {}
best_models = {}

for model in models:
    print(f"Optimizing {model}...")
    best_params[model] = optimize_model(model, Xtrain, Ytrain)
    print(f"Best parameters for {model}: {best_params[model]}")
    best_models[model] = train_best_model(model, best_params[model], Xtrain, Ytrain)

# Training and evaluating the best model
for model_name, model in best_models.items():
    Ypred = model.predict(Xtest)
    accuracy = accuracy_score(Ytest, Ypred)
    print(f"Accuracy of {model_name}: {accuracy}")

# Return the best models
best_models


[I 2024-05-17 15:59:18,289] A new study created in memory with name: no-name-60a64647-b69b-45e1-9ea9-112973b41159


Optimizing svm...


[I 2024-05-17 15:59:29,482] Trial 0 finished with value: 0.5225480174612477 and parameters: {'C': 0.001187780281131538, 'kernel': 'rbf', 'gamma': 'auto'}. Best is trial 0 with value: 0.5225480174612477.
[I 2024-05-17 15:59:37,556] Trial 1 finished with value: 0.5709777722656056 and parameters: {'C': 0.030061526722114755, 'kernel': 'poly', 'degree': 5, 'gamma': 'auto'}. Best is trial 1 with value: 0.5709777722656056.


Best parameters for svm: {'C': 0.030061526722114755, 'kernel': 'poly', 'degree': 5, 'gamma': 'auto'}


[I 2024-05-17 15:59:41,743] A new study created in memory with name: no-name-db39a4ea-d113-4516-b7f5-8b1d230548a7


...

[I 2024-05-17 16:01:40,957] A new study created in memory with name: no-name-cb0fe2d3-f534-427b-8b9d-d8b5b524dc74


Optimizing random_forest...


[I 2024-05-17 16:01:46,664] Trial 0 finished with value: 0.988960606824477 and parameters: {'n_estimators': 148, 'max_depth': 15, 'min_samples_split': 6, 'min_samples_leaf': 1}. Best is trial 0 with value: 0.988960606824477.
[I 2024-05-17 16:01:53,458] Trial 1 finished with value: 0.985490177115342 and parameters: {'n_estimators': 184, 'max_depth': 17, 'min_samples_split': 10, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.988960606824477.


Best parameters for random_forest: {'n_estimators': 148, 'max_depth': 15, 'min_samples_split': 6, 'min_samples_leaf': 1}
Accuracy of svm: 1.0
Accuracy of xgboost: 1.0
Accuracy of logistic_regression: 0.7299338999055713
Accuracy of random_forest: 1.0


{'svm': SVC(C=0.030061526722114755, degree=5, gamma='auto', kernel='poly'),
 'xgboost': XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, device=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric=None, feature_types=None,
               gamma=None, grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=0.1690636482579104,
               max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=8, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               multi_strategy=None, n_estimators=200, n_jobs=None,
               num_parallel_tree=None, random_state=None, ...),
 'logistic_regression': LogisticRegression(C=29.567169611610247, max_iter=10000),
 'random_forest': RandomForestClassifier(max_depth=

In [20]:
best_models


{'svm': SVC(C=0.030061526722114755, degree=5, gamma='auto', kernel='poly'),
 'xgboost': XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, device=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric=None, feature_types=None,
               gamma=None, grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=0.1690636482579104,
               max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
               max_delta_step=None, max_depth=8, max_leaves=None,
               min_child_weight=None, missing=nan, monotone_constraints=None,
               multi_strategy=None, n_estimators=200, n_jobs=None,
               num_parallel_tree=None, random_state=None, ...),
 'logistic_regression': LogisticRegression(C=29.567169611610247, max_iter=10000),
 'random_forest': RandomForestClassifier(max_depth=

## Theory 1 

1. Score both Main & Co as two seperate customers

bs = fit(bs)

ws = fit(ws)

In [ ]:

# Example usage:
# Replace with your actual data
# X_train, X_test, y_train, y_test = np.array(...)

models = ['svm', 'xgboost', 'lightgbm', 'random_forest']
best_params = {}
best_models = {}

for model in models:
    print(f"Optimizing {model}...")
    best_params[model] = optimize_model(model, X_train, y_train)
    print(f"Best parameters for {model}: {best_params[model]}")
    best_models[model] = train_best_model(model, best_params[model], X_train, y_train)

# Training and evaluating the best model
for model_name, model in best_models.items():
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy of {model_name}: {accuracy}")

# Return the best models
best_models


Optimizing svm...


NameError: name 'optimize_model' is not defined